In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import matplotlib.pyplot as plt

model = YOLO("yolov8n.pt")

In [4]:
from pathlib import Path

def count_bikes_in_video(video_path, show_every_nth_frame=0):
    """
    Zählt Velos in jedem Frame eines Videos mit YOLO.
    
    video_path: Pfad zur Videodatei (str oder Path)
    show_every_nth_frame: wenn >0, zeigt jeden n-ten Frame mit Bounding Boxes
    """
    video_path = Path(video_path)
    if not video_path.exists():
        raise FileNotFoundError(f"Video nicht gefunden: {video_path}")
    
    cap = cv2.VideoCapture(str(video_path))
    frame_counts = []
    frame_indices = []
    
    frame_idx = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # YOLO-Inferenz
        results = model.predict(frame, verbose=False)
        boxes = results[0].boxes
        classes = boxes.cls.cpu().numpy() if boxes is not None else []
        
        # COCO-Klassen-ID für Fahrrad ist 1, für Motorrad 3 (je nach Version, zur Sicherheit ausgeben)
        bike_ids = [1]   # erstmal nur Fahrräder
        
        bike_count = sum(cls in bike_ids for cls in classes)
        
        frame_counts.append(bike_count)
        frame_indices.append(frame_idx)
        
        # Optional visualisieren
        if show_every_nth_frame > 0 and frame_idx % show_every_nth_frame == 0:
            plotted = results[0].plot()
            plt.figure(figsize=(6,4))
            plt.imshow(plotted[:, :, ::-1])
            plt.title(f"Frame {frame_idx} – Velos: {bike_count}")
            plt.axis("off")
            plt.show()
        
        frame_idx += 1
    
    cap.release()
    return np.array(frame_indices), np.array(frame_counts)


In [ ]:
frame_indices, bike_counts = count_bikes_in_video("data/velo_park.mp4", show_every_nth_frame=50)

In [5]:
plt.figure(figsize=(10,4))
plt.plot(frame_indices, bike_counts)
plt.xlabel("Frame")
plt.ylabel("Anzahl Velos")
plt.title("Velobelegung im Video über die Zeit")
plt.grid(True)
plt.show()

print("Maximale Anzahl Velos im Video:", bike_counts.max())
print("Durchschnittliche Velobelegung:", bike_counts.mean())

NameError: name 'frame_indices' is not defined

<Figure size 1000x400 with 0 Axes>